In [294]:
import csv

def read_csv(filename):
    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        rows = [row for row in reader]
    return rows


rooms = read_csv('data/Public Bump - Rooms.csv')
bumpers = read_csv('data/Public Bump - Bumpers.csv')
t = read_csv('data/Public Bump - Time slots.csv')


In [295]:
d_gender = { "M": 1, "F": 2}
c = {"SUITE": 2, "SINGLE": 1, "DOUBLE":2, "TRIPLE": 3, "QUAD": 4}


# AJUSTAR LA PUNTAUCION DE LAS HABITACIONES A LA IDEAL
d_p = {"SUITE": 49, "SINGLE": 50, "DOUBLE": 22, "TRIPLE": 10, "QUAD": 1}

class Room:
    
    id = 0
    tipo = ''
    genero = 0  # 1 (Male), 2 (Female), 3 (Mixed)
    inquilinos = []
    capacidad = 0
    preferencias = []
    p = 0

    def __init__(self, id, tipo, capacidad):
        self.inquilinos = list()
        self.id = id
        self.tipo = tipo
        self.capacidad = capacidad
        self.p = d_p[tipo]
    
    def lleno(self):
        return self.capacidad == len(self.inquilinos)
    
    def espacio(self):
        return self.capacidad - len(self.inquilinos)
    
    def add_inquilino(self, inquilino):
        if self.lleno():
            return False
        self.inquilinos.append(inquilino)
        self.p = self.p/2
        if(self.lleno()):
            self.p = 0
        return True
    
    def set_genero(self, genero):
        g = d_gender[genero]
        if self.genero == 0:
            self.genero = g
        elif self.genero != g:
            self.genero = 3

    def toString(self):
        return f'[POINTS: {self.p}] Room {self.id} - {self.tipo} - {self.genero} - {self.espacio()} / {self.capacidad} '
    
    def __lt__(self, other):
        return self.p > other.p
    

    
class Bumper:
    id = 0
    seniority = 0

    def __init__(self, id, s):
        self.id = id
        self.seniority = s


In [296]:
estado = { "AVAILABLE": True, "TAKEN": False }
all_rooms = dict()

for room in rooms:
    id = room['Room']
    tipo = room['Room Type']
    gender = room['Gender']
    dn = room['Deposit Number']
    capacity = c[tipo]

    if(id in all_rooms):
        if dn == "":
            continue
        dn = int(dn)
        all_rooms[id].add_inquilino(dn)
        all_rooms[id].set_genero(gender)

    else:
        all_rooms[id] = Room(id, tipo, capacity)
        if dn != "":
            dn = int(dn)
            all_rooms[id].add_inquilino(dn)
            all_rooms[id].set_genero(gender)





#sorted_tuples = sorted(all_rooms.items(), key=lambda x: getattr(x[1], 'p'), reverse=True)
#sorted_dict = dict(sorted_tuples)


#for room in sorted_dict.values():
#    print(room.toString())



In [297]:
coopers = list()
for bumper in bumpers:
    id = bumper['Deposit Number']
    s = bumper['Seniority']
    if s == "":
        s = 0
    s = int(s)
    coopers.append(Bumper(id, s))

#for bumper in coopers:
#    print(f'Bumper {bumper.id} - {bumper.seniority}')

In [298]:
d_times = dict()
for time in t:
    d_times[time['Deposit Number/Group Number']] = time['Time']


In [299]:
class HHH:
    floor = 0
    rooms = []
    def __init__(self, floor):
        self.floor = floor
        self.rooms = list()

    def toString(self):
        return [ x.id for x in self.rooms]
    


floors = list()

for i in range(1, 8):
    floors.append(HHH(i))

for room in all_rooms:
    id = 0
    try:
        id = int(room)
    except:
        continue
    floor = int(id/100)
    floors[floor-1].rooms.append(all_rooms[room])


for floor in floors:
    floor.rooms.sort(key=lambda x: x.id)



In [300]:
floor = 2
d_gen = {0: "UNDEFINED" ,1: "MALE", 2: "FEMALE", 3: "MIXED"}
def print_floor(floor):
    s = ""
    for i in floors[floor].rooms:
        id_str = f'Room {i.id}'.ljust(10)
        tipo_str = f'TYPE: {i.tipo}'.ljust(15)
        genero_str = f'GENDER: {d_gen[i.genero]}'.ljust(20)
        espacio_str = f'SPACES AVAILABLES: {i.espacio()} / {i.capacidad}'.ljust(30)
        s += (id_str + tipo_str + genero_str + espacio_str + "\n")
    return s

In [301]:
s = ""
for i in range(1, 8):
    s += f'FLOOR {i}\n'
    s+='-----------------------------------------------------------------------------------\n'
    s+=(print_floor(i-1))
    s+=('-----------------------------------------------------------------------------------\n')

with open('output/HHH.txt', 'w') as f:
    # Write some text to the file
    f.write(s)


In [302]:
from queue import PriorityQueue

priority_queue = PriorityQueue()

for room in all_rooms.values():
    priority_queue.put((-room.p, room))



In [303]:
results = dict()
from tqdm import tqdm

s = ""
for cooper in tqdm(coopers):
    a_room = priority_queue.get()[1]
    s += a_room.toString() + "\n"
    a_room.add_inquilino(cooper.id)
    priority_queue.put((-a_room.p, a_room))
    results[cooper.id] = a_room

    


100%|██████████| 228/228 [00:00<00:00, 113764.13it/s]


In [304]:
with open('output/result.txt', 'w') as f:
    # Write some text to the file
    for i in results:
        f.write(f'{i} - {results[i].toString()}\n ')


with open('output/orden.txt', 'w') as f:
    # Write some text to the file
    for i in results:
        f.write(s)

with open('output/rooms.txt', 'w') as f:
    # Write some text to the file
    sorted_tuples = sorted(all_rooms.items(), key=lambda x: getattr(x[1], 'p'), reverse=True)
    for i in sorted_tuples:
        f.write(f'{i[1].toString()} \n')


In [305]:
DEPOSIT_NUMBER = "27565" #CHANGE THIS VALUE

print("--------------------------------------------------------------------------")
print("PREDICTION LAUNCHED FOR " + str(DEPOSIT_NUMBER) + " DEPOSIT NUMBER")
print("--------------------------------------------------------------------------")
print(f'ROOM TYPE: {results[DEPOSIT_NUMBER].tipo} - WITH {results[DEPOSIT_NUMBER].espacio()} OF {results[DEPOSIT_NUMBER].capacidad} SPACES AVAILABLES')
print("--------------------------------------------------------------------------")
print("BUMP TIME: " + d_times[DEPOSIT_NUMBER])
print("--------------------------------------------------------------------------\n")


--------------------------------------------------------------------------
PREDICTION LAUNCHED FOR 27565 DEPOSIT NUMBER
--------------------------------------------------------------------------
ROOM TYPE: DOUBLE - WITH 0 OF 2 SPACES AVAILABLES
--------------------------------------------------------------------------
BUMP TIME: 7:30 PM on 10/17/23
--------------------------------------------------------------------------



In [306]:
print(all_rooms['422'].toString())

[POINTS: 0] Room 422 - TRIPLE - 2 - 0 / 3 


In [307]:
#Calculate total coopers
all_coopers = 0

for room in all_rooms.values():
    all_coopers += len(room.inquilinos)

print("ALL COOPERS: " + str(all_coopers))
print("BUMPERS: " + str(len(coopers)))




ALL COOPERS: 408
BUMPERS: 228
